# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4889, uptime 0:00:17
mini_beamline                    RUNNING   pid 4890, uptime 0:00:17
random_walk                      RUNNING   pid 4891, uptime 0:00:17
random_walk_horiz                RUNNING   pid 4892, uptime 0:00:17
random_walk_vert                 RUNNING   pid 4893, uptime 0:00:17
simple                           RUNNING   pid 4894, uptime 0:00:17
thermo_sim                       RUNNING   pid 4895, uptime 0:00:17
trigger_with_pc                  RUNNING   pid 4896, uptime 0:00:17


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 1     Time: 2020-05-15 21:00:45
Persistent Unique Scan ID: '5c34c51e-2fc5-4c43-9fee-e79aa9124cb5'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 21:00:45.6 |     -1.000 |     100973 |


|         2 | 21:00:45.7 |     -0.500 |     102996 |
|         3 | 21:00:45.7 |      0.000 |     104793 |
|         4 | 21:00:45.7 |      0.500 |     103618 |
|         5 | 21:00:45.7 |      1.000 |     102017 |
+-----------+------------+------------+------------+


generator scan ['5c34c51e'] (scan num: 1)





('5c34c51e-2fc5-4c43-9fee-e79aa9124cb5',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/5c34c51e-2fc5-4c43-9fee-e79aa9124cb5-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 2     Time: 2020-05-15 21:00:46
Persistent Unique Scan ID: '31de7446-ee38-4b50-87eb-56bf46512401'


New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 21:00:46.2 |          -1 | 1131191858 |


|         2 | 21:00:46.3 |          -0 | 1130762288 |


|         3 | 21:00:46.4 |           0 | 1128536033 |


|         4 | 21:00:46.6 |           0 | 1125051807 |


|         5 | 21:00:46.7 |           1 | 1118970329 |


+-----------+------------+-------------+------------+
generator scan ['31de7446'] (scan num: 2)





('31de7446-ee38-4b50-87eb-56bf46512401',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-15 21:00:47
Persistent Unique Scan ID: 'ce4e421d-3ae4-4c90-b158-fab615cbbe17'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 21:00:47.2 | 1090458753 |


|         2 | 21:00:48.2 | 1053428560 |


|         3 | 21:00:49.2 | 1095365048 |


...

|        28 | 21:01:14.2 | 1131528987 |


|        29 | 21:01:15.2 | 1087341564 |


|        30 | 21:01:16.2 | 1053670131 |


+-----------+------------+------------+
generator count ['ce4e421d'] (scan num: 3)





('ce4e421d-3ae4-4c90-b158-fab615cbbe17',)